# 🎯 Probabilidade Condicional e Teorema de Bayes: Atualizando a Crença

## Módulo 3 - Estatística para IA

### Pedro Nunes Guth

---

E aí, pessoal! Bora para mais uma jornada no mundo da estatística? 🚀

Hoje vamos falar sobre um dos conceitos mais **poderosos** da estatística: a **Probabilidade Condicional** e o famoso **Teorema de Bayes**!

Tá, mas o que isso tem a ver com IA? TUDO! Desde filtros de spam até diagnósticos médicos, o Teorema de Bayes está por toda parte. É literalmente a matemática por trás de "atualizar nossas crenças" com novas evidências.

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/estatística-para-ia-modulo-03_img_01.png)

In [ ]:
# Imports necessários para nossa jornada bayesiana!
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import warnings

warnings.filterwarnings('ignore')

# Configurações para gráficos mais bonitos
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print("🎉 Bibliotecas carregadas! Bora descobrir os segredos de Bayes!")

## 🤔 O que é Probabilidade Condicional?

**Probabilidade Condicional** é a probabilidade de um evento acontecer, dado que outro evento já aconteceu.

Pensa assim: você tá no shopping e vê que tá chovendo lá fora. Qual a probabilidade de você ter esquecido o guarda-chuva em casa, **dado que** tá chovendo?

A notação matemática é:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

Onde:
- $P(A|B)$ = Probabilidade de A acontecer dado que B já aconteceu
- $P(A \cap B)$ = Probabilidade de A E B acontecerem juntos
- $P(B)$ = Probabilidade de B acontecer

**Dica do Pedro:** Lê como "Probabilidade de A dado B". O símbolo | significa "dado que" ou "condicionado a".

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/estatística-para-ia-modulo-03_img_02.png)

In [ ]:
# Vamos criar um exemplo prático de probabilidade condicional
# Cenário: Estudantes aprovados em matemática e física

# Dados simulados de 1000 estudantes
np.random.seed(42)
n_estudantes = 1000

# Probabilidades base
prob_matematica = 0.7  # 70% passam em matemática
prob_fisica = 0.6      # 60% passam em física
prob_ambas = 0.5       # 50% passam em ambas

# Simulando os dados
passou_matematica = np.random.choice([0, 1], size=n_estudantes, p=[1-prob_matematica, prob_matematica])
passou_fisica = np.random.choice([0, 1], size=n_estudantes, p=[1-prob_fisica, prob_fisica])

# Ajustando para ter correlação realista
# Se passou em matemática, tem mais chance de passar em física
for i in range(n_estudantes):
    if passou_matematica[i] == 1:
        passou_fisica[i] = np.random.choice([0, 1], p=[0.2, 0.8])
    else:
        passou_fisica[i] = np.random.choice([0, 1], p=[0.7, 0.3])

# Criando DataFrame
df_estudantes = pd.DataFrame({
    'matematica': passou_matematica,
    'fisica': passou_fisica
})

print("📊 Dados dos estudantes:")
print(df_estudantes.head(10))
print(f"\n📈 Total de estudantes: {len(df_estudantes)}")

In [ ]:
# Calculando probabilidades condicionais na prática

# Probabilidades básicas
prob_mat = df_estudantes['matematica'].mean()
prob_fis = df_estudantes['fisica'].mean()
prob_ambas = ((df_estudantes['matematica'] == 1) & (df_estudantes['fisica'] == 1)).mean()

print("🎯 PROBABILIDADES BÁSICAS:")
print(f"P(Matemática) = {prob_mat:.3f}")
print(f"P(Física) = {prob_fis:.3f}")
print(f"P(Matemática E Física) = {prob_ambas:.3f}")

# Probabilidade condicional: P(Física | Matemática)
estudantes_mat = df_estudantes[df_estudantes['matematica'] == 1]
prob_fisica_dado_mat = estudantes_mat['fisica'].mean()

print("\n🔥 PROBABILIDADES CONDICIONAIS:")
print(f"P(Física | Matemática) = {prob_fisica_dado_mat:.3f}")
print(f"Usando a fórmula: P(Física | Matemática) = {prob_ambas/prob_mat:.3f}")

# Probabilidade condicional: P(Matemática | Física)
estudantes_fis = df_estudantes[df_estudantes['fisica'] == 1]
prob_mat_dado_fisica = estudantes_fis['matematica'].mean()

print(f"P(Matemática | Física) = {prob_mat_dado_fisica:.3f}")
print(f"Usando a fórmula: P(Matemática | Física) = {prob_ambas/prob_fis:.3f}")

print("\n💡 Viu a diferença? A ordem importa na probabilidade condicional!")

In [ ]:
# Visualizando probabilidades condicionais
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Tabela de contingência
tabela_contingencia = pd.crosstab(df_estudantes['matematica'], df_estudantes['fisica'], 
                                 rownames=['Matemática'], colnames=['Física'])

# Heatmap da tabela de contingência
sns.heatmap(tabela_contingencia, annot=True, fmt='d', cmap='Blues', 
            ax=axes[0], cbar_kws={'label': 'Número de Estudantes'})
axes[0].set_title('Tabela de Contingência\n(Contagem de Estudantes)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Passou em Física')
axes[0].set_ylabel('Passou em Matemática')

# Probabilidades condicionais visualizadas
prob_data = {
    'P(Física|Matemática=1)': [prob_fisica_dado_mat, 1-prob_fisica_dado_mat],
    'P(Física|Matemática=0)': [df_estudantes[df_estudantes['matematica'] == 0]['fisica'].mean(),
                              1-df_estudantes[df_estudantes['matematica'] == 0]['fisica'].mean()]
}

x = np.arange(2)
width = 0.35

axes[1].bar(x - width/2, prob_data['P(Física|Matemática=1)'], width, 
           label='P(Física|Matemática=1)', alpha=0.8)
axes[1].bar(x + width/2, prob_data['P(Física|Matemática=0)'], width, 
           label='P(Física|Matemática=0)', alpha=0.8)

axes[1].set_xlabel('Resultado em Física')
axes[1].set_ylabel('Probabilidade')
axes[1].set_title('Probabilidades Condicionais\nP(Física | Matemática)', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(['Reprovou', 'Passou'])
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("📊 Liiindo! Vê como a probabilidade de passar em física muda dependendo se passou em matemática!")

## 🧠 O Teorema de Bayes: A Fórmula Mágica

Agora vem a parte mais **sensacional**! O Teorema de Bayes é uma extensão da probabilidade condicional que nos permite "virar o jogo".

**Cenário:** Você tem uma crença inicial sobre algo (probabilidade a priori) e recebe uma nova evidência. O Teorema de Bayes te ajuda a atualizar sua crença (probabilidade a posteriori).

### A Fórmula Sagrada:

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

Ou na versão mais completa:

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B|A) \cdot P(A) + P(B|\neg A) \cdot P(\neg A)}$$

Onde:
- $P(A|B)$ = **Posteriori** (o que queremos descobrir)
- $P(B|A)$ = **Verossimilhança** (quão provável é a evidência dado A)
- $P(A)$ = **Priori** (nossa crença inicial)
- $P(B)$ = **Evidência** (probabilidade marginal)

**Dica do Pedro:** Pensa assim: "Eu tinha uma opinião, recebi uma informação nova, agora preciso atualizar minha opinião de forma inteligente!"

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/estatística-para-ia-modulo-03_img_03.png)

In [ ]:
# Exemplo clássico: Teste médico
# Vamos simular um teste para detectar uma doença

def teorema_bayes(prob_priori, prob_verossimilhanca, prob_evidencia_total):
    """
    Calcula a probabilidade posteriori usando o Teorema de Bayes
    
    Args:
        prob_priori: P(A) - Probabilidade a priori
        prob_verossimilhanca: P(B|A) - Probabilidade da evidência dado A
        prob_evidencia_total: P(B) - Probabilidade total da evidência
    
    Returns:
        Probabilidade a posteriori P(A|B)
    """
    return (prob_verossimilhanca * prob_priori) / prob_evidencia_total

# Dados do problema médico
print("🏥 CENÁRIO: Teste para detectar uma doença rara")
print("="*50)

# Probabilidades conhecidas
prob_doenca = 0.001  # 0.1% da população tem a doença (muito rara!)
prob_teste_positivo_com_doenca = 0.99  # Teste tem 99% de sensibilidade
prob_teste_positivo_sem_doenca = 0.02  # 2% de falsos positivos

print(f"📊 P(Doença) = {prob_doenca:.3f} ({prob_doenca*100:.1f}% da população)")
print(f"🎯 P(Teste+|Doença) = {prob_teste_positivo_com_doenca:.3f} (Sensibilidade)")
print(f"⚠️  P(Teste+|Sem Doença) = {prob_teste_positivo_sem_doenca:.3f} (Falsos positivos)")

# Calculando P(Teste+) - probabilidade total de teste positivo
prob_teste_positivo = (prob_teste_positivo_com_doenca * prob_doenca + 
                      prob_teste_positivo_sem_doenca * (1 - prob_doenca))

print(f"\n🧮 P(Teste+) = {prob_teste_positivo:.4f}")

# Aplicando Bayes: P(Doença|Teste+)
prob_doenca_dado_teste_positivo = teorema_bayes(
    prob_doenca, 
    prob_teste_positivo_com_doenca, 
    prob_teste_positivo
)

print(f"\n🎯 RESULTADO USANDO BAYES:")
print(f"P(Doença|Teste+) = {prob_doenca_dado_teste_positivo:.4f} ({prob_doenca_dado_teste_positivo*100:.2f}%)")
print(f"\n🤯 SURPRESA! Mesmo com teste positivo, só tem {prob_doenca_dado_teste_positivo*100:.2f}% de chance de ter a doença!")
print("   Isso acontece porque a doença é muito rara!")

In [ ]:
# Visualizando o impacto da prevalência no resultado de Bayes

# Testando diferentes prevalências da doença
prevalencias = np.logspace(-4, -1, 50)  # De 0.01% a 10%
prob_posteriores = []

for prev in prevalencias:
    prob_teste_pos_total = (prob_teste_positivo_com_doenca * prev + 
                           prob_teste_positivo_sem_doenca * (1 - prev))
    
    prob_post = teorema_bayes(prev, prob_teste_positivo_com_doenca, prob_teste_pos_total)
    prob_posteriores.append(prob_post)

# Criando o gráfico
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Gráfico 1: Probabilidade posterior vs prevalência
ax1.semilogx(prevalencias * 100, np.array(prob_posteriores) * 100, 'b-', linewidth=3)
ax1.axhline(y=50, color='r', linestyle='--', alpha=0.7, label='50% (Linha de Referência)')
ax1.axvline(x=0.1, color='g', linestyle='--', alpha=0.7, label='Prevalência Original (0.1%)')
ax1.set_xlabel('Prevalência da Doença (%)', fontsize=12)
ax1.set_ylabel('P(Doença | Teste+) (%)', fontsize=12)
ax1.set_title('Como a Prevalência Afeta o\nResultado do Teorema de Bayes', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Gráfico 2: Barras mostrando o caso específico
categorias = ['P(Doença)\n(Priori)', 'P(Doença|Teste+)\n(Posteriori)']
valores = [prob_doenca * 100, prob_doenca_dado_teste_positivo * 100]
cores = ['lightblue', 'darkblue']

bars = ax2.bar(categorias, valores, color=cores, alpha=0.8)
ax2.set_ylabel('Probabilidade (%)', fontsize=12)
ax2.set_title('Atualizando a Crença\ncom o Teorema de Bayes', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

# Adicionando valores nas barras
for bar, valor in zip(bars, valores):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height + 0.1,
            f'{valor:.3f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📈 Liiindo! Vê como a prevalência (probabilidade a priori) influencia MUITO o resultado!")
print("💡 Dica do Pedro: Em doenças raras, mesmo testes precisos podem ter baixo valor preditivo positivo!")

## 📧 Aplicação Clássica: Filtro de Spam Bayesiano

Bora para o exemplo mais **clássico** e útil do Teorema de Bayes: filtrar spam! 🎯

A ideia é simples:
1. Temos emails que são spam e não-spam
2. Cada palavra tem uma probabilidade de aparecer em spam vs não-spam
3. Quando chega um email novo, usamos Bayes para calcular a probabilidade de ser spam

### Como funciona:

Para cada palavra $w_i$ no email:

$$P(\text{Spam}|w_1, w_2, ..., w_n) = \frac{P(w_1, w_2, ..., w_n|\text{Spam}) \cdot P(\text{Spam})}{P(w_1, w_2, ..., w_n)}$$

**Assumindo independência** (Naive Bayes):

$$P(\text{Spam}|\text{palavras}) = \frac{\prod_{i=1}^{n} P(w_i|\text{Spam}) \cdot P(\text{Spam})}{P(\text{palavras})}$$

**Dica do Pedro:** "Naive" porque assume que as palavras são independentes (sabemos que não são, mas funciona bem na prática!).

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/estatística-para-ia-modulo-03_img_04.png)

In [ ]:
# Criando um dataset de emails simulado para demonstrar o filtro de spam

# Emails de spam (características típicas)
emails_spam = [
    "GANHE DINHEIRO RÁPIDO! CLIQUE AQUI AGORA! OFERTA LIMITADA!",
    "Você ganhou um prêmio! Clique para resgatar seu dinheiro grátis!",
    "PROMOÇÃO IMPERDÍVEL! Dinheiro fácil! Ganhe muito agora!",
    "Oferta exclusiva! Clique aqui e ganhe dinheiro sem trabalhar!",
    "URGENTE! Seu prêmio expira hoje! Clique agora para ganhar!",
    "Dinheiro grátis! Não perca essa oportunidade única! Clique já!",
    "PARABÉNS! Você foi selecionado para ganhar R$ 10.000! Clique!",
    "Oferta por tempo limitado! Ganhe dinheiro rápido e fácil!",
    "ATENÇÃO! Promoção especial! Clique e receba seu prêmio!",
    "Oportunidade única! Ganhe dinheiro trabalhando em casa!"
]

# Emails legítimos (ham)
emails_ham = [
    "Reunião agendada para amanhã às 14h na sala de conferências.",
    "Obrigado pelo seu trabalho excelente no projeto. Parabéns pela dedicação.",
    "Lembrete: prazo de entrega do relatório é sexta-feira.",
    "Oi, tudo bem? Vamos almoçar juntos hoje? Que tal aquele restaurante?",
    "Confirmação do seu pedido. Produto será entregue em 3 dias úteis.",
    "Feliz aniversário! Desejo muito sucesso e felicidade sempre!",
    "Reunião cancelada devido ao feriado. Reagendaremos para próxima semana.",
    "Relatório mensal está pronto. Favor revisar e aprovar quando possível.",
    "Convite para apresentação do novo produto na próxima segunda-feira.",
    "Obrigado pela sua participação na reunião. Suas ideias foram valiosas."
]

# Criando DataFrame
emails_data = []

# Adicionando emails spam
for email in emails_spam:
    emails_data.append({'texto': email.lower(), 'spam': 1})

# Adicionando emails ham
for email in emails_ham:
    emails_data.append({'texto': email.lower(), 'spam': 0})

df_emails = pd.DataFrame(emails_data)

print("📧 DATASET DE EMAILS CRIADO:")
print(f"Total de emails: {len(df_emails)}")
print(f"Emails spam: {df_emails['spam'].sum()}")
print(f"Emails legítimos: {len(df_emails) - df_emails['spam'].sum()}")

print("\n📝 EXEMPLOS:")
print("\nSPAM:")
print(f'"{
f"}"')
print("\nLEGÍTIMO:")
print(f'"{
f"}"')

# Mostrando estatísticas básicas
print(f"\n📊 Distribuição: {df_emails['spam'].value_counts().to_dict()}")

In [ ]:
# Implementando um classificador Naive Bayes do zero!
from collections import defaultdict, Counter
import math

class NaiveBayesSpam:
    def __init__(self):
        self.palavras_spam = defaultdict(int)
        self.palavras_ham = defaultdict(int)
        self.total_palavras_spam = 0
        self.total_palavras_ham = 0
        self.prob_spam = 0
        self.prob_ham = 0
        self.vocabulario = set()
        
    def preprocessar_texto(self, texto):
        """Remove pontuação e divide em palavras"""
        import re
        # Remove pontuação e converte para minúsculas
        texto = re.sub(r'[^\w\s]', '', texto.lower())
        return texto.split()
    
    def treinar(self, emails, labels):
        """Treina o classificador com emails e seus rótulos"""
        print("🧠 Treinando o classificador Naive Bayes...")
        
        # Contando emails spam vs ham
        total_emails = len(emails)
        spam_count = sum(labels)
        ham_count = total_emails - spam_count
        
        # Probabilidades a priori
        self.prob_spam = spam_count / total_emails
        self.prob_ham = ham_count / total_emails
        
        print(f"📊 P(Spam) = {self.prob_spam:.3f}")
        print(f"📊 P(Ham) = {self.prob_ham:.3f}")
        
        # Contando palavras em cada classe
        for email, label in zip(emails, labels):
            palavras = self.preprocessar_texto(email)
            self.vocabulario.update(palavras)
            
            if label == 1:  # Spam
                for palavra in palavras:
                    self.palavras_spam[palavra] += 1
                    self.total_palavras_spam += 1
            else:  # Ham
                for palavra in palavras:
                    self.palavras_ham[palavra] += 1
                    self.total_palavras_ham += 1
        
        print(f"📚 Vocabulário: {len(self.vocabulario)} palavras únicas")
        print(f"📝 Total palavras spam: {self.total_palavras_spam}")
        print(f"📝 Total palavras ham: {self.total_palavras_ham}")
    
    def calcular_probabilidade_palavra(self, palavra, classe):
        """Calcula P(palavra|classe) com suavização de Laplace"""
        if classe == 'spam':
            count = self.palavras_spam[palavra]
            total = self.total_palavras_spam
        else:
            count = self.palavras_ham[palavra]
            total = self.total_palavras_ham
        
        # Suavização de Laplace (adiciona 1 ao numerador e vocabulário ao denominador)
        return (count + 1) / (total + len(self.vocabulario))
    
    def classificar(self, email):
        """Classifica um email como spam ou ham"""
        palavras = self.preprocessar_texto(email)
        
        # Calculando log-probabilidades (para evitar underflow)
        log_prob_spam = math.log(self.prob_spam)
        log_prob_ham = math.log(self.prob_ham)
        
        for palavra in palavras:
            log_prob_spam += math.log(self.calcular_probabilidade_palavra(palavra, 'spam'))
            log_prob_ham += math.log(self.calcular_probabilidade_palavra(palavra, 'ham'))
        
        # Convertendo de volta para probabilidades (aproximação)
        if log_prob_spam > log_prob_ham:
            return 1, math.exp(log_prob_spam) / (math.exp(log_prob_spam) + math.exp(log_prob_ham))
        else:
            return 0, math.exp(log_prob_ham) / (math.exp(log_prob_spam) + math.exp(log_prob_ham))

# Testando nosso classificador
classificador = NaiveBayesSpam()
classificador.treinar(df_emails['texto'].tolist(), df_emails['spam'].tolist())

print("\n✅ Classificador treinado com sucesso!")

In [ ]:
# Testando o classificador com novos emails

emails_teste = [
    "Ganhe muito dinheiro! Oferta imperdível! Clique agora!",
    "Oi, como vai? Vamos nos encontrar para o almoço?",
    "PRÊMIO URGENTE! Você ganhou! Clique para resgatar!",
    "Reunião de trabalho cancelada. Reagendaremos em breve.",
    "Oferta limitada! Dinheiro grátis! Não perca!"
]

print("🧪 TESTANDO O CLASSIFICADOR:")
print("="*60)

for i, email in enumerate(emails_teste, 1):
    predicao, confianca = classificador.classificar(email)
    classe = "SPAM" if predicao == 1 else "HAM"
    
    print(f"\n📧 Email {i}: \"{email}\"")
    print(f"🎯 Classificação: {classe} (Confiança: {confianca:.3f})")
    
    if classe == "SPAM":
        print("🚨 ⚠️  Este email foi identificado como SPAM!")
    else:
        print("✅ 📬 Este email parece legítimo.")

print("\n💡 Dica do Pedro: Vê como o classificador capturou os padrões das palavras!")
print("   Palavras como 'ganhe', 'dinheiro', 'clique' aumentam a probabilidade de spam!")

In [ ]:
# Analisando as palavras mais "spam" e "ham"

# Calculando "spamicidade" de cada palavra
palavras_importantes = {}

for palavra in classificador.vocabulario:
    if len(palavra) > 2:  # Ignorando palavras muito pequenas
        prob_spam = classificador.calcular_probabilidade_palavra(palavra, 'spam')
        prob_ham = classificador.calcular_probabilidade_palavra(palavra, 'ham')
        
        # Razão spam/ham (maior = mais spam)
        razao_spam = prob_spam / prob_ham if prob_ham > 0 else float('inf')
        
        if razao_spam != 1:  # Ignorando palavras neutras
            palavras_importantes[palavra] = {
                'razao_spam': razao_spam,
                'prob_spam': prob_spam,
                'prob_ham': prob_ham,
                'count_spam': classificador.palavras_spam[palavra],
                'count_ham': classificador.palavras_ham[palavra]
            }

# Ordenando por razão spam/ham
palavras_ordenadas = sorted(palavras_importantes.items(), 
                           key=lambda x: x[1]['razao_spam'], reverse=True)

print("🔍 ANÁLISE DE PALAVRAS MAIS INDICATIVAS:")
print("="*70)

print("\n🚨 TOP 10 PALAVRAS MAIS 'SPAM':")
for palavra, stats in palavras_ordenadas[:10]:
    print(f"{palavra:15} | Razão: {stats['razao_spam']:6.2f} | "
          f"P(spam): {stats['prob_spam']:5.3f} | P(ham): {stats['prob_ham']:5.3f}")

print("\n✅ TOP 10 PALAVRAS MAIS 'HAM':")
for palavra, stats in palavras_ordenadas[-10:]:
    print(f"{palavra:15} | Razão: {stats['razao_spam']:6.2f} | "
          f"P(spam): {stats['prob_spam']:5.3f} | P(ham): {stats['prob_ham']:5.3f}")

print("\n💡 Dica do Pedro: Razão > 1 = mais provável em spam, Razão < 1 = mais provável em ham!")

In [ ]:
# Visualizando o desempenho do classificador

# Testando em todo o dataset (só para demonstração)
predicoes = []
confiancas = []

for email in df_emails['texto']:
    pred, conf = classificador.classificar(email)
    predicoes.append(pred)
    confiancas.append(conf)

df_emails['predicao'] = predicoes
df_emails['confianca'] = confiancas

# Calculando métricas
acuracia = (df_emails['spam'] == df_emails['predicao']).mean()

print(f"🎯 Acurácia no dataset: {acuracia:.3f} ({acuracia*100:.1f}%)")

# Criando visualizações
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Matriz de Confusão
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(df_emails['spam'], df_emails['predicao'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
           xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
axes[0,0].set_title('Matriz de Confusão', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Predição')
axes[0,0].set_ylabel('Real')

# 2. Distribuição de Confiança
spam_conf = df_emails[df_emails['spam'] == 1]['confianca']
ham_conf = df_emails[df_emails['spam'] == 0]['confianca']

axes[0,1].hist(spam_conf, alpha=0.7, label='Spam Real', bins=10, color='red')
axes[0,1].hist(ham_conf, alpha=0.7, label='Ham Real', bins=10, color='blue')
axes[0,1].set_xlabel('Confiança da Predição')
axes[0,1].set_ylabel('Frequência')
axes[0,1].set_title('Distribuição de Confiança', fontsize=14, fontweight='bold')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# 3. Palavras mais spam
top_spam_words = [palavra for palavra, _ in palavras_ordenadas[:8]]
top_spam_ratios = [stats['razao_spam'] for _, stats in palavras_ordenadas[:8]]

axes[1,0].barh(range(len(top_spam_words)), top_spam_ratios, color='red', alpha=0.7)
axes[1,0].set_yticks(range(len(top_spam_words)))
axes[1,0].set_yticklabels(top_spam_words)
axes[1,0].set_xlabel('Razão Spam/Ham')
axes[1,0].set_title('Palavras Mais Indicativas de Spam', fontsize=14, fontweight='bold')
axes[1,0].grid(True, alpha=0.3)

# 4. Evolução das probabilidades
prob_priori_spam = df_emails['spam'].mean()
prob_posteriori_spam = df_emails[df_emails['predicao'] == 1]['spam'].mean() if len(df_emails[df_emails['predicao'] == 1]) > 0 else 0

categorias = ['P(Spam)\nPriori', 'P(Spam|Palavras)\nPosteriori']
probs = [prob_priori_spam * 100, prob_posteriori_spam * 100]

bars = axes[1,1].bar(categorias, probs, color=['lightcoral', 'darkred'], alpha=0.8)
axes[1,1].set_ylabel('Probabilidade (%)')
axes[1,1].set_title('Atualização Bayesiana', fontsize=14, fontweight='bold')
axes[1,1].grid(True, alpha=0.3, axis='y')

for bar, prob in zip(bars, probs):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + 1,
                  f'{prob:.1f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 Liiindo! Vê como o Teorema de Bayes nos ajuda a atualizar nossas crenças!")

## 🚀 Comparando com Scikit-learn

Bora ver como nossa implementação se compara com a versão profissional do scikit-learn!

**Dica do Pedro:** É sempre bom implementar do zero para entender, mas na prática usamos bibliotecas otimizadas!

In [ ]:
# Comparando com sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Criando pipeline do sklearn
pipeline_sklearn = Pipeline([
    ('vectorizer', CountVectorizer(lowercase=True, stop_words=None)),
    ('classifier', MultinomialNB())
])

# Treinando
pipeline_sklearn.fit(df_emails['texto'], df_emails['spam'])

# Fazendo predições
predicoes_sklearn = pipeline_sklearn.predict(df_emails['texto'])
probabilidades_sklearn = pipeline_sklearn.predict_proba(df_emails['texto'])[:, 1]

# Comparando resultados
acuracia_sklearn = (df_emails['spam'] == predicoes_sklearn).mean()
acuracia_nossa = (df_emails['spam'] == df_emails['predicao']).mean()

print("🥊 COMPARAÇÃO NOSSA IMPLEMENTAÇÃO vs SKLEARN:")
print("="*55)
print(f"📊 Acurácia Nossa Implementação: {acuracia_nossa:.3f} ({acuracia_nossa*100:.1f}%)")
print(f"📊 Acurácia Sklearn:             {acuracia_sklearn:.3f} ({acuracia_sklearn*100:.1f}%)")

# Testando nos emails de teste
print("\n🧪 TESTANDO NOS EMAILS NOVOS:")
print("="*50)

for i, email in enumerate(emails_teste, 1):
    # Nossa implementação
    pred_nossa, conf_nossa = classificador.classificar(email)
    
    # Sklearn
    pred_sklearn = pipeline_sklearn.predict([email])[0]
    prob_sklearn = pipeline_sklearn.predict_proba([email])[0]
    conf_sklearn = max(prob_sklearn)
    
    print(f"\n📧 Email {i}: \"{email[:50]}...\"")
    print(f"   Nossa:   {'SPAM' if pred_nossa == 1 else 'HAM':4} (conf: {conf_nossa:.3f})")
    print(f"   Sklearn: {'SPAM' if pred_sklearn == 1 else 'HAM':4} (conf: {conf_sklearn:.3f})")
    print(f"   Acordo:  {'✅' if pred_nossa == pred_sklearn else '❌'}")

print("\n💡 Dica do Pedro: Pequenas diferenças são normais devido a implementações ligeiramente diferentes!")

## 🎯 Exercício Prático 1: Diagnóstico Médico

Agora é sua vez! Vamos aplicar o Teorema de Bayes em um cenário médico.

**Cenário:** Um teste para detectar COVID-19 tem:
- Sensibilidade: 95% (detecta corretamente 95% dos casos positivos)
- Especificidade: 98% (detecta corretamente 98% dos casos negativos)
- Prevalência na população: 2%

**Sua missão:**
1. Calcule P(COVID | Teste+) usando Bayes
2. Varie a prevalência de 0.1% a 10% e veja como muda o resultado
3. Explique por que isso é importante para políticas de saúde pública

In [ ]:
# 🎯 EXERCÍCIO 1: Complete o código abaixo

def diagnostico_covid_bayes(sensibilidade, especificidade, prevalencia):
    """
    Calcula P(COVID | Teste+) usando Teorema de Bayes
    
    Args:
        sensibilidade: P(Teste+ | COVID) 
        especificidade: P(Teste- | Sem COVID)
        prevalencia: P(COVID)
    
    Returns:
        P(COVID | Teste+)
    """
    # TODO: Implementar o cálculo de Bayes
    # P(Teste+ | Sem COVID) = 1 - especificidade
    prob_teste_pos_sem_covid = None  # COMPLETE AQUI
    
    # P(Teste+) = P(Teste+|COVID) * P(COVID) + P(Teste+|Sem COVID) * P(Sem COVID)
    prob_teste_positivo = None  # COMPLETE AQUI
    
    # P(COVID | Teste+) usando Bayes
    prob_covid_dado_teste_pos = None  # COMPLETE AQUI
    
    return prob_covid_dado_teste_pos

# Parâmetros do teste
sensibilidade = 0.95
especificidade = 0.98
prevalencia_base = 0.02

# TODO: Calcule o resultado para a prevalência base
resultado = None  # COMPLETE AQUI

print(f"🏥 DIAGNÓSTICO COVID-19:")
print(f"Sensibilidade: {sensibilidade*100}%")
print(f"Especificidade: {especificidade*100}%")
print(f"Prevalência: {prevalencia_base*100}%")
print(f"\n🎯 P(COVID | Teste+) = {resultado:.4f} ({resultado*100:.2f}%)")

# TODO: Teste com diferentes prevalências e crie um gráfico
# prevalencias = np.linspace(0.001, 0.1, 100)
# resultados = [diagnostico_covid_bayes(sensibilidade, especificidade, p) for p in prevalencias]
# Faça um gráfico mostrando como a prevalência afeta o resultado!

## 🎯 Exercício Prático 2: Melhorando o Filtro de Spam

Vamos aprimorar nosso filtro de spam!

**Sua missão:**
1. Adicione mais emails ao dataset (pelo menos 5 de cada tipo)
2. Implemente uma função para remover stop words (a, de, para, etc.)
3. Teste diferentes técnicas de pré-processamento
4. Compare a performance antes e depois das melhorias

In [ ]:
# 🎯 EXERCÍCIO 2: Aprimore o filtro de spam

# TODO: Adicione mais emails ao dataset
emails_spam_extras = [
    # ADICIONE PELO MENOS 5 EMAILS DE SPAM AQUI
]

emails_ham_extras = [
    # ADICIONE PELO MENOS 5 EMAILS LEGÍTIMOS AQUI
]

# TODO: Implemente remoção de stop words
stop_words_pt = ['a', 'de', 'do', 'da', 'em', 'para', 'com', 'por', 'e', 'o', 'os', 'as', 'um', 'uma']

def remover_stop_words(texto, stop_words):
    """
    Remove stop words do texto
    """
    palavras = texto.split()
    # TODO: IMPLEMENTE A REMOÇÃO DE STOP WORDS
    palavras_filtradas = None  # COMPLETE AQUI
    return ' '.join(palavras_filtradas)

# TODO: Crie uma versão melhorada da classe NaiveBayesSpam
class NaiveBayesSpamMelhorado(NaiveBayesSpam):
    def __init__(self, usar_stop_words=True):
        super().__init__()
        self.usar_stop_words = usar_stop_words
        self.stop_words = stop_words_pt
    
    def preprocessar_texto(self, texto):
        # TODO: IMPLEMENTE O PRÉ-PROCESSAMENTO MELHORADO
        # 1. Limpar pontuação
        # 2. Converter para minúsculas  
        # 3. Remover stop words se usar_stop_words=True
        # 4. Retornar lista de palavras
        pass

# TODO: Teste e compare os resultados
print("🚧 Implemente as melhorias e teste o desempenho!")
print("💡 Dica: Compare acurácia antes e depois das melhorias")

## 🧠 Conceitos Avançados: Por que Bayes Funciona?

### A Matemática por Trás da Magia

O Teorema de Bayes é **poderoso** porque:

1. **Combina evidências**: Une informação prévia com nova evidência
2. **É ótimo**: Matematicamente provado como classificador ótimo sob certas condições
3. **É robusto**: Funciona bem mesmo com dados imperfeitos
4. **É interpretável**: Podemos entender **por que** uma decisão foi tomada

### Conexão com IA Moderna

- **Redes Bayesianas**: Grafos que modelam dependências probabilísticas
- **Inferência Variacional**: Aproximações para Bayes em problemas complexos  
- **Aprendizado Bayesiano**: Quantifica incerteza nas predições
- **GANs**: Usam princípios bayesianos implicitamente

**Dica do Pedro:** Bayes está por toda parte na IA, mesmo quando não percebemos!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/estatística-para-ia-modulo-03_img_05.png)

In [ ]:
# Demonstrando a evolução das crenças com múltiplas evidências

def atualizar_crenca_sequencial(crenca_inicial, evidencias, verossimilhancas):
    """
    Mostra como Bayes atualiza crenças sequencialmente
    """
    crenca_atual = crenca_inicial
    historico = [crenca_atual]
    
    print(f"🎯 Crença inicial: {crenca_atual:.4f}")
    
    for i, (evidencia, verossim) in enumerate(zip(evidencias, verossimilhancas), 1):
        # Aplicando Bayes
        # P(H|E) = P(E|H) * P(H) / P(E)
        # Assumindo P(E) = P(E|H)*P(H) + P(E|¬H)*P(¬H)
        # E P(E|¬H) = 1 - P(E|H) para simplificar
        
        prob_evidencia = (verossim * crenca_atual + 
                         (1 - verossim) * (1 - crenca_atual))
        
        crenca_atual = (verossim * crenca_atual) / prob_evidencia
        historico.append(crenca_atual)
        
        print(f"📊 Evidência {i}: {evidencia}")
        print(f"   Verossimilhança: {verossim:.3f}")
        print(f"   Nova crença: {crenca_atual:.4f}")
        print()
    
    return historico

# Exemplo: Investigação criminal
print("🕵️ INVESTIGAÇÃO CRIMINAL - ATUALIZANDO SUSPEITAS:")
print("="*60)

crenca_inicial = 0.1  # 10% de chance inicial de ser culpado
evidencias = [
    "Estava no local do crime",
    "Não tem álibi", 
    "Tinha motivo",
    "Impressão digital encontrada",
    "Testemunha o viu fugindo"
]
verossimilhancas = [0.8, 0.7, 0.6, 0.95, 0.9]  # P(evidência | culpado)

historico = atualizar_crenca_sequencial(crenca_inicial, evidencias, verossimilhancas)

# Visualizando a evolução
plt.figure(figsize=(12, 6))
plt.plot(range(len(historico)), historico, 'bo-', linewidth=3, markersize=8)
plt.xlabel('Número de Evidências', fontsize=12)
plt.ylabel('P(Culpado | Evidências)', fontsize=12)
plt.title('Evolução da Crença com Teorema de Bayes\n(Investigação Criminal)', 
         fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)

# Adicionando anotações
for i, (prob, evidencia) in enumerate(zip(historico[1:], evidencias), 1):
    plt.annotate(f'{prob:.3f}', 
                xy=(i, prob), 
                xytext=(10, 10), 
                textcoords='offset points',
                fontweight='bold')

plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.7, label='Limiar 50%')
plt.legend()
plt.tight_layout()
plt.show()

print(f"🎯 Resultado final: {historico[-1]:.4f} ({historico[-1]*100:.2f}% de certeza)")
print("\n💡 Vê como cada evidência atualiza nossa crença de forma matemática!")

## 📈 Conexão com os Próximos Módulos

O que aprendemos aqui vai ser **fundamental** para os próximos módulos:

### 🔗 Módulo 4 - Regressão Linear:
- Interpretação probabilística da regressão
- Maximum Likelihood Estimation (MLE)
- Incerteza nas predições

### 🔗 Módulo 6 - Regressão Logística:
- Regressão Logística como classificador Bayesiano
- Odds ratio e probabilidades
- Interpretação das probabilidades de classe

### 🔗 Módulo 8 - Testes de Hipótese:
- Probabilidades condicionais em testes estatísticos
- P-valores como probabilidades condicionais
- Erro Tipo I e Tipo II

**Dica do Pedro:** Bayes é a base de MUITA coisa em estatística e IA. Dominar esses conceitos vai te dar uma vantagem enorme!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/estatística-para-ia-modulo-03_img_06.png)

## 🎯 Resumo: O que Aprendemos Hoje

### ✅ Conceitos Dominados:

1. **Probabilidade Condicional**: P(A|B) = Como eventos se relacionam
2. **Teorema de Bayes**: A fórmula para atualizar crenças com evidências
3. **Classificação Bayesiana**: Aplicação prática em filtros de spam
4. **Naive Bayes**: Assumindo independência para simplificar cálculos
5. **Interpretabilidade**: Entendendo POR QUE um modelo toma decisões

### 🧮 Fórmulas Essenciais:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

$$P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}$$

### 💡 Insights Importantes:

- **Prevalência importa**: Em doenças raras, testes podem ter baixo valor preditivo
- **Evidências se acumulam**: Cada nova informação atualiza nossa crença
- **Simplicidade funciona**: Naive Bayes é "ingênuo" mas muito eficaz
- **Interpretabilidade é poder**: Conseguimos explicar as decisões do modelo

### 🚀 Próximos Passos:

No **Módulo 4**, vamos mergulhar na **Regressão Linear** e ver como ela se conecta com os conceitos probabilísticos que aprendemos hoje!

**Dica do Pedro:** Continue praticando! Bayes está por toda parte - desde filtros de email até carros autônomos. Quanto mais você usar, mais natural vai ficar!

![](/Users/pedroguth/Downloads/Projetos/Book Maker/5-Imagens/estatística-para-ia-modulo-03_img_07.png)